In [1]:
from collections import Counter, defaultdict, deque
from copy import deepcopy
from operator import add
import pickle
import string

import pandas as pd
from sqlalchemy import create_engine
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from pyspark import SparkConf
import numpy as np
import Levenshtein
import pyspark
from pyspark import Row
from pyspark.sql import SparkSession
import seaborn as sns

%matplotlib inline
sns.set(color_codes=True)

# Initialise sparkContext

In [2]:
spark = SparkSession.builder  \
    .master("local[28]") \
    .appName("func-dedup") \
    .config("spark.executor.memory", "20gb") \
    .config("spark.driver.memory", "100G") \
    .config("spark.driver.maxResultSize", "10G") \
    .config("spark.local.dir", "/storage/egor/tmp/spark_tmp") \
    .getOrCreate()

# Read datasets

In [3]:
data_loc_dedupl = "/storage/egor/tmp/1M_file_java_uast_extracted_func_dedupl"
rdd = (spark.read.parquet(data_loc_dedupl).rdd
       .persist(pyspark.StorageLevel.DISK_ONLY))
print("Number of rows in dataset: {}".format(rdd.count()))

Number of rows in dataset: 7743853


# Mapping of internal types to characters
* number of internal types allow us to replace them with characters
* Levenshtein works only with strings (standard diff lib gives wrong results in some cases)

In [4]:
def uast2sequence(root):
    sequence = []
    nodes = defaultdict(deque)
    stack = [root]
    nodes[id(root)].extend(root.children)
    while stack:
        if nodes[id(stack[-1])]:
            child = nodes[id(stack[-1])].popleft()
            nodes[id(child)].extend(child.children)
            stack.append(child)
        else:
            sequence.append(stack.pop())
    return sequence


def internal_type(row):
    from bblfsh import Node
    parse_uast = Node.FromString
    uast = parse_uast(row.uast[0])
    return list(set(node.internal_type for node in uast2sequence(uast)))

internal_types = rdd.flatMap(internal_type).distinct().collect()

type2char = {int_type: char for int_type, char in zip(internal_types, string.printable)}
print("Number of internal typers: {}".format(len(type2char)))
print(type2char)

def uast2chars(uast):
    from bblfsh import Node
    parse_uast = Node.FromString
    try:
        uast = parse_uast(uast)
    except:
        pass
    return "".join([type2char[node.internal_type] for node in uast2sequence(uast)])

Number of internal typers: 84
{'SuperFieldAccess': '+', 'Dimension': 'X', 'SimpleName': '.', 'SuperMethodInvocation': '=', 'SimpleType': 'z', 'MethodRefParameter': 'J', 'InstanceofExpression': 'W', 'NullLiteral': 'E', 'BooleanLiteral': '4', 'PostfixExpression': 'w', 'MemberValuePair': '5', 'TagElement': '3', 'FieldDeclaration': '@', 'NameQualifiedType': 'j', 'MethodRef': 'H', 'VariableDeclarationExpression': '%', 'TypeParameter': ':', 'WhileStatement': 'b', 'ArrayCreation': '#', 'InfixExpression': 'U', 'MemberRef': 'h', 'ParenthesizedExpression': '/', 'Javadoc': 'g', 'VariableDeclarationFragment': 'k', 'SwitchCase': 'Z', 'MethodInvocation': '?', 'Initializer': '9', 'IfStatement': '2', 'ReturnStatement': '0', 'ClassInstanceCreation': '(', 'SwitchStatement': '6', 'CastExpression': 'd', 'UnionType': 'p', 'LambdaExpression': 'f', 'CreationReference': ';', 'IntersectionType': 'S', 'Modifier': 'B', 'PrefixExpression': 'Q', 'TypeDeclarationStatement': 'A', 'ArrayAccess': '8', 'EnhancedForStat

In [5]:
def is_content_bad(content):
    """
    Check if it's only function definition
    """
    return not content.strip().endswith("}")


def filter_n_nodes(rdd, n_nodes_min, n_nodes_max, n_samples=5):
    print("min {}, max {}".format(n_nodes_min, n_nodes_max))
    print("-+=" * 20)
    return (rdd.filter(lambda r: n_nodes_min < len(uast2chars(r.uast[0])) <= n_nodes_max)
            .filter(lambda r: not is_content_bad(r.content)).take(n_samples))

In [6]:
res = filter_n_nodes(rdd, 0, 5, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 0, max 5
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
IoUrlInfo_lnx() {}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public ListNode() {
		
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
private Util() {}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
Dsv_fld_parser_bry() {}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
Xow_domain_crt_itm__none() {}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=


In [7]:
res = filter_n_nodes(rdd, 5, 10, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 5, max 10
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public void begin() {
        GLES20.glUseProgram(programHandle);
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public String getPassword() {
		return password;
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public Bry_bfr Get_b512() {return mkr_b512.Get();}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public boolean Exit_code_pass() {return exit_code == Exit_pass;}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public Io_url Trg() {return trg;}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=


In [8]:
res = filter_n_nodes(rdd, 10, 15, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 10, max 15
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
    protected void onResume() {
        super.onResume();
        oglView.onResume();
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
    protected void onResume() {
        super.onResume();
        oglView.onResume();
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public Bry_bfr Add_byte_colon()			{return Add_byte(Byte_ascii.Colon);}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public static byte[] Ary(byte... ary) {return ary;}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public static String To_str(char c) {return String.valueOf(c);}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=


In [9]:
res = filter_n_nodes(rdd, 15, 20, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 15, max 20
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
	protected void onStart() {
		// TODO Auto-generated method stub
		super.onStart();
		Log.i(TAG, "B --> onStart()");
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public synchronized Collection<User> getAllUsers() {
    return Collections.unmodifiableCollection(users.values());
  }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public OGLView(Context context, AttributeSet attrs) {
        super(context, attrs);
        init();
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Autowired
  public SimpleRepository(JdbcTemplate jdbcTemplate) {
    super();
    this.jdbcTemplate = jdbcTemplate;
  }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
/** @param name the name of the attribute
     * @return whether the attribute is available in the shader */
    public boolean hasAttribute(String name) {
        return attributes.containsKey(name);
 

In [10]:
res = filter_n_nodes(rdd, 20, 25, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 20, max 25
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public void unBoundServiceMethod()
	{	if(mIsBound)
	{
		this.unbindService(mServiceConnection);
		Toast.makeText(getApplicationContext(), "unBound Service", Toast.LENGTH_LONG).show();
	}
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
	protected void onCreate(Bundle savedInstanceState) {
		super.onCreate(savedInstanceState);
		setContentView(R.layout.dashboard);
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
blic void Test_new_u8(String raw, byte[] expd)			{Tfds.Eq_ary(expd, Bry_.new_u8(raw));}

-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public int compareTo(Object obj)	{return CompareAble_.Compare_obj(raw, ((Io_url)obj).raw);}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public static void Eq_list(List_adp lhs, List_adp rhs, TfdsEqListItmStr xtoStr)			{Eq_list_wkr(lhs, rhs, xtoStr, EmptyStr);}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+

In [11]:
res = filter_n_nodes(rdd, 25, 30, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 25, max 30
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@DELETE
  @Path("/{id}")
  public Response deleteUser(@PathParam("id") int identifier) {
    userService.deleteUser(identifier);
    return Response.ok().build();
  }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
	protected void onCreate (Bundle savedInstanceState) {
		super.onCreate(savedInstanceState);
		AndroidApplicationConfiguration config = new AndroidApplicationConfiguration();
		initialize(new AccelerometerAxesGame(), config);
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
void tst_Add_long_fixed(long val, int digits, String expd) {Tfds.Eq(expd, String_.new_u8(bb.Add_long_fixed(val, digits).To_bry()));}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@gplx.Internal protected void ctor_IoItmBase_url(Io_url url) {this.url = url; this.name = url.NameAndExt();}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Test  public void CmdDlm_DoNotS

In [12]:
res = filter_n_nodes(rdd, 30, 35, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 30, max 35
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
  public void onMessage(Message message) {
    try {
      simpleService.processText(message.getBody(String.class));
    } catch (Throwable t) {
      log.error("Error during message reception", t);
    }
  }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public static ShortBuffer newShortBuffer (int numShorts) {
        ByteBuffer buffer = ByteBuffer.allocateDirect(numShorts * 2);
        buffer.order(ByteOrder.nativeOrder());
        return buffer.asShortBuffer();
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
/**
	 * Creates a new instance of {@link EntityPlayerSP}.
	 */
	public static EntityPlayerSP makePlayer() {
		return new EntityPlayerSP(WDL.minecraft, WDL.worldClient,
				WDL.thePlayer.connection, WDL.thePlayer.getStatFileWriter());
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public SubSortThread(int[] in, int l, int r) {
		this.in = in;
	

In [13]:
res = filter_n_nodes(rdd, 35, 40, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 35, max 40
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Test   public void Math() {
		fxt.Exec__parse(Gfh_utl.Replace_apos("<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/596f8baf206a81478afd4194b44138715dc1a05c' class='mwe-math-fallback-image-inline' aria-hidden='true' style='vertical-align: -2.005ex; width:16.822ex; height:6.176ex;' alt='R_{H}=a\\left({\\frac {m}{3M}}\\right)^{\\frac {1}{3}}'>"))
			.Test__html(Gfh_utl.Replace_apos("<img src='xowa:/math/596f8baf206a81478afd4194b44138715dc1a05c.svg' class='mwe-math-fallback-image-inline' aria-hidden='true' style='vertical-align: -2.005ex; width:16.822ex; height:6.176ex;' alt='R_{H}=a\\left({\\frac {m}{3M}}\\right)^{\\frac {1}{3}}'>"))
			.Test__fsdb(fxt.Make__fsdb(Bool_.Y, Bool_.Y, "596f8baf206a81478afd4194b44138715dc1a05c.svg", -1, -1, -1));
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public static Dbui_tbl_itm__bmk get_or_new(Xoa_app app, Xoud_bmk_itm_tbl db_tbl) {
		if (I 

In [14]:
res = filter_n_nodes(rdd, 40, 45, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 40, max 45
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
private int fetchAttributeLocation(String name) {

        int location = attributes.get(name);
        if(location == -1) {
            location = GLES20.glGetAttribLocation(programHandle, name);
            if(location != -1) {
                attributes.put(name, location);
            }
        }
        return location;
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Test
  public void testSi() throws InterruptedException, IOException {
    // GIVEN

    // WHEN
    boolean result = wrapperService.processText(MESSAGE_TEXT);

    // THEN
    TestWriteRepository testWriteRepository =
        (TestWriteRepository) writeRepository;
    Assert.assertEquals(testWriteRepository.getMessage(), MESSAGE_TEXT
        + " transformed");
    Assert.assertEquals(result, true);
  }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
private void tst_MatchAtCurExact(String srcStr, Object expd) 

In [15]:
res = filter_n_nodes(rdd, 45, 50, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 45, max 50
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
/** Sets the uniform with the given name.
     *
     * @param name the name of the uniform
     * @param color r, g, b and a as the first through fourth values respectively */
    public void setUniformColor (String name, int color) {
        setUniformf(name, Color.red(color) / 255.0f, Color.green(color) / 255.0f, Color.blue(color) / 255.0f, Color.alpha(color) / 255.0f);
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
/**
     * Determines whether the specified coordinates are contained in this 
     * <code>Polygon</code>.
     * @param x the specified x coordinate to be tested
     * @param y the specified y coordinate to be tested
     * @return  <code>true</code> if this <code>Polygon</code> contains
     * 		the specified coordinates, (<i>x</i>,&nbsp;<i>y</i>);  
     * 		<code>false</code> otherwise.
     * @see #contains(double, double)
     * @deprecated As of JDK version 1.1,
     

In [16]:
res = filter_n_nodes(rdd, 50, 55, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 50, max 55
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Bean
  public Job prepareStrongTeaJob(JobBuilderFactory jobBuilderFactory,
      @Qualifier("boilWaterStep") Step boilWaterStep,
      @Qualifier("addTeaStep") Step addTeaStep,
      @Qualifier("addWaterStep") Step addWaterStep) {
    return jobBuilderFactory.get("prepareStrongTeaJob")
        .start(boilWaterStep)
        .next(addTeaStep)
        .next(addTeaStep)
        .next(addWaterStep)
        .build();
  }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public void Queue(Gfo_invk invk, String invk_key, Object... args) {
		Gfo_async_cmd_itm cmd = cmd_mkr.Get(invk, invk_key, args);
		synchronized (queue) {
			queue.Add(cmd);
		}
		synchronized (running) {
			if (running.Val_n()) {
				running.Val_y_();
				Thread_adp_.Start_by_key(Invk_run, this, Invk_run);
			}
		}
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public void Update_all(byte[] ptr_ttl, int w, int h, byte orig

In [17]:
res = filter_n_nodes(rdd, 55, 60, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 55, max 60
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
/**
	 * Get the update from the VariableEditorUIUpdateThread
	 */
	public void update(Observable o, Object arg) {
		if (o == VariableEditorUIUpdateThread.getInstance()) {
			// if this component is no longer usable, remove it as an Observer.
			if (!isDisplayable())
				VariableEditorUIUpdateThread.getInstance().deleteObserver(this);
			else if (!variable.evaluate().toParseableString().equals(
					displayedValue))
				if (!hasFocus())
					updateWithCurrentVariableValue();
		} else if (arg == variable)// if the update is coming directly from the
									// variable
			updateWithCurrentVariableValue();
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
static LinkedListNode findFirstCommonNode(LinkedListNode l1,
			LinkedListNode l2) {
		HashSet<Object> h = new HashSet<Object>();
		while (l1 != null) {
			h.add(l1);
			l1 = l1.next;
		}
		while (l2 != null) {
			if (h.contains(l2))
				return l2;


In [18]:
res = filter_n_nodes(rdd, 60, 65, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 60, max 65
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public DaoSession(SQLiteDatabase db, IdentityScopeType type, Map<Class<? extends AbstractDao<?, ?>>, DaoConfig>
            daoConfigMap) {
        super(db);

        todoDaoConfig = daoConfigMap.get(TodoDao.class).clone();
        todoDaoConfig.initIdentityScope(type);

        todoDao = new TodoDao(todoDaoConfig, this);

        registerDao(Todo.class, todoDao);
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override public java.io.OutputStream Wrap_stream(java.io.OutputStream stream) {		
		try {return new org.tukaani.xz.XZOutputStream(stream, new org.tukaani.xz.LZMA2Options(org.tukaani.xz.LZMA2Options.PRESET_DEFAULT));}
		catch (Exception e) {throw Err_.new_exc(e, "io", "failed to open xz stream");}
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public Xop_fxt	Init_page_update(Xowe_wiki wiki, String ttl, String txt) {
		Xoa_ttl page_ttl = Xoa_ttl.Parse(wiki, Bry_.new_u8

In [19]:
res = filter_n_nodes(rdd, 65, 70, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 65, max 70
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
static void bld_(IptCfg cfg, IptBndsOwner box, Gfo_invk invk, String bnd_key, GfoMsg m, IptEventType ev_type, IptArg... ipt) {
		IptCfgItm itm = cfg.GetOrDefaultArgs(bnd_key, m, ipt);
		IptBnd bnd = IptBnd_invk.new_(box, invk, itm, ev_type);
		cfg.Owners_add(bnd_key, box);
		box.IptBnds().Add(bnd);
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override public int Chk_hook(Tst_mgr mgr, String path, Object actl_obj, int err) {
		Xot_invk_tkn actl = (Xot_invk_tkn)actl_obj;
		if (name_tkn != null) err += mgr.Tst_sub_obj(name_tkn, actl.Name_tkn(), path + "." + "name", err);
		err += mgr.Tst_sub_ary(args, Args_xtoAry(actl), path + "." + "args", err);
		return err;
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
    public void execute(Player admin, String... params) {
        Player target = null;
        VisibleObject creature = admin.getTarget();

        if (creature ==

In [20]:
res = filter_n_nodes(rdd, 70, 75, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 70, max 75
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
private void addTiersHash2(ITier tier, int varName1, int varName2)
    {
        long key = varName1 < varName2 ? (long)varName1 << 21 | varName2 : (long)varName2 << 21 | varName1;
        
        ObjectArrayList tiers = (ObjectArrayList) tiersHash2.get(key);
        
        if (tiers == null)
        {
            tiersHash2.put(key, new ObjectArrayList(new ITier[] {tier}));
        }
        else
        {
            tiers.add(tier);
        }
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public Object Invk(GfsCtx ctx, int ikey, String k, GfoMsg m) {
		if		(ctx.Match(k, Invk_async_bgn))				Download();
		else if	(ctx.Match(k, Invk_owner))					return owner;
		else if	(ctx.Match(k, Invk_src_))					src = m.ReadStr("v");
		else if	(ctx.Match(k, Invk_trg_))					trg = Bry_fmtr_eval_mgr_.Eval_url(url_eval_mgr, m.ReadBry("v"));
		else	return Gfo_invk_.Rv_unhandled;
		return this;
	}
-+=-+=-+=-+

In [21]:
res = filter_n_nodes(rdd, 75, 80, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 75, max 80
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Test(groups = "maventests")
  public void testGetUserDetails() throws IOException, InterruptedException {
    // GIVEN
    Process process = new ProcessExecutor().execute("0303-ws-boot-service.jar");
    try {

      FunctionRetryHandler<String, UserDetailsResponse> retryHandler =
          new FunctionRetryHandler<>();

      // WHEN
      UserDetailsResponse userDetails =
          retryHandler.retry(wsClient::getUserDetails, "lubos.krnac@gmail.com",
              RETRY_TIMEOUT);

      // THEN
      assertEquals(userDetails.getFirstName(), "Lubos");
      assertEquals(userDetails.getLastName(), "Krnac");
    } finally {
      process.destroyForcibly();
      process.waitFor();
    }
  }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public ImageAdp Extract_image(int src_x, int src_y, int src_w, int src_h, int trg_w, int trg_h) {
		if (this == ImageAdp_.Null) return ImageAdp_.Null; // TODO_OLD:

In [22]:
res = filter_n_nodes(rdd, 80, 85, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 80, max 85
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
private long setProgress() {
		if (mPlayer == null || mDragging)
			return 0;

		long position = mPlayer.getCurrentPosition();
		long duration = mPlayer.getDuration();
		if (duration > 0) {
			long pos = 1000L * position / duration;
			mProgress.setProgress((int) pos);
		}
		int percent = mPlayer.getBufferPercentage();
		mProgress.setSecondaryProgress(percent * 10);

		mDuration = duration;

		mEndTime.setText(StringUtils.generateTime(mDuration));
		mCurrentTime.setText(StringUtils.generateTime(position));

		return position;
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
    public void execute(Player player, String... params) {
        if (params == null || params.length < 1) {
            onFail(player, null);
            return;
        }

        String address = params[0];
        boolean result = BannedMacManager.getInstance().unbanAddress(address,
                "uban;mac=" +

In [23]:
res = filter_n_nodes(rdd, 85, 90, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 85, max 90
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
private int linkProgram(int program) {
        if(program == -1)
            return -1;

        GLES20.glAttachShader(program, vertexShaderHandle);
        GLES20.glAttachShader(program, fragmentShaderHandle);
        GLES20.glLinkProgram(program);

        int[] linked = new int[1];
        GLES20.glGetProgramiv(program, GLES20.GL_LINK_STATUS, linked, 0);
        if(linked[0] == GLES20.GL_FALSE) {
            String infoLog = GLES20.glGetProgramInfoLog(program);
            log += infoLog;
            GLES20.glDeleteProgram(program);
            return  -1;
        }
        return program;
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Bean
    public LocalContainerEntityManagerFactoryBean entityManagerFactory() {
        logger.debug("\n\n************ {} ************\n\n",
                getDatabaseDialect().getCanonicalName());
        HibernateJpaVendorAdapter vendorAdapter = new Hib

In [24]:
res = filter_n_nodes(rdd, 90, 95, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 90, max 95
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
    public boolean onOptionsItemSelected(MenuItem item) {
        ActiveAndroid.beginTransaction();
        try {
            List<Todo> ended = new Select()
                    .from(Todo.class)
                    .where(Todo.COLUMN_STATUS + "=?",
                            Integer.toString(TodoDaoHelper.STATUS_END))
                    .execute();
            for (Todo todo : ended) {
                todo.setStatus(TodoDaoHelper.STATUS_ARCHIVE);
                todo.save();
            }
            ActiveAndroid.setTransactionSuccessful();
        } finally {
            ActiveAndroid.endTransaction();
        }

        // 画面更新
        if (mTodoFragment != null) {
            mTodoFragment.reload();
        }
        if (mHistoryFragment != null) {
            mHistoryFragment.reload();
        }

        return true;
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public void

In [25]:
res = filter_n_nodes(rdd, 95, 100, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 95, max 100
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
private void fetchAttributes() {

        attributes.clear();

        IntBuffer params = IntBuffer.allocate(1);
        IntBuffer type = IntBuffer.allocate(1);
        GLES20.glGetProgramiv(programHandle, GLES20.GL_ACTIVE_ATTRIBUTES, params);
        int numAttributes = params.get(0);

        for(int i=0; i<numAttributes; i++) {
            params.compact();
            params.put(0, 1);
            type.clear();
            String name = GLES20.glGetActiveAttrib(programHandle, i, params, type);
            int location = GLES20.glGetAttribLocation(programHandle, name);
            attributes.put(name, location);
        }
    }
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
private static void tracePath(int i, int j, int[][] mem)  {
		if(i < 0 || j < 0) return;
		if( (i<1?0:mem[i-1][j]) > (j<1?0:mem[i][j-1]) ) {
			tracePath(i-1, j, mem);
			System.out.println("(" + i + "," + j + ")");
		} els

In [26]:
res = filter_n_nodes(rdd, 100, 105, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 100, max 105
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public static void nonRec4(BTreeNode current) {
		Stack<StackInfo> stack = new Stack<StackInfo>();
		while (true) {
			while (current != null) {		
				stack.push(new StackInfo(current, false));
				current = current.left;
			}
			if(stack.isEmpty()) break;
			StackInfo tmp = stack.pop();
			if(tmp.flag == false) {
				tmp.flag = true;
				stack.push(tmp);
				current = tmp.node.right;
			} else {
				System.out.print(tmp.node.data+" ");
				//just to continue with pop statement
				current = null;
			}
		}
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
/**
	 * {@inheritDoc}
	 */
	@Override
	public boolean updateAccountTime(final int accountId, final AccountTime accountTime) {
		return DB.insertUpdate("REPLACE INTO account_time (account_id, last_active, expiration_time, "
			+ "session_duration, accumulated_online, accumulated_rest, penalty_end) values " + "(?,?,?,?,?,?,?)",
			new IUStH() {



In [27]:
res = filter_n_nodes(rdd, 105, 110, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 105, max 110
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public static void initConfig(Context context) {
		// if true , it means , have load the infos, so need not to load again .
		if(init) {
			return;
		}
		try {
			//read the configure file to the memory
			InputStream inputStream =  context.getResources().openRawResource(R.raw.config);
//			InputStream inputStream =  AppConfigs.class.getResourceAsStream("/anim/"+config);
			Properties properties  = new Properties();
			properties.load(inputStream);
			debug = Boolean.valueOf(properties.getProperty("debug"));
			L.e("Debug:"+debug);
			persistLog = Boolean.valueOf(properties.getProperty("persistLog"));
			L.e("persistLog:"+persistLog);

			mainCategoruUrl = properties.getProperty("mainCategoruUrl");
			L.e("mainCategoruUrl:"+mainCategoruUrl);
			init = true;
		} catch (IOException e) {
			e.printStackTrace();
		}
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
public static int distinctPaths4(i

In [28]:
res = filter_n_nodes(rdd, 110, 115, n_samples=5)
for row in res:
    print(row.content)
    print("-+=" * 20)

min 110, max 115
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
private void Load_itm_meta(Bry_bfr tmp_bfr, Xoedit_nde_hash itm_list) {
		Xogui_nde_iter iter = Xogui_nde_iter.New_sql(itm_list);
		while (iter.Move_next()) {
			String sql = Db_sql_.Make_by_fmt(String_.Ary
			( "SELECT  *"
			, "FROM    cfg_itm i"
			, "WHERE   i.itm_key IN ({0})"
			), iter.To_sql_in_key()
			);

			Db_rdr rdr = db_app.Conn().Stmt_sql(sql).Exec_select__rls_auto();
			while (rdr.Move_next()) {
				String key = rdr.Read_str("itm_key");
				Xoedit_itm edit_itm = (Xoedit_itm)itm_list.Get_by_or_fail(key);
				String dflt = dflt_mgr.Get_or(key, rdr.Read_str("itm_dflt"));
				edit_itm.Load_by_meta(tmp_bfr, rdr.Read_str("itm_type"), dflt, rdr.Read_str("itm_html_atrs"), rdr.Read_str("itm_html_cls"));
			}
		}
	}
-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=-+=
@Override
    public boolean onDialogEvent(QuestEnv env) {
        Player player = env.getPlayer();
        QuestState qs = playe